# Triple Collocation

Triple collocation (TC) is a method for approximating the error standard deviation of a property measured by 3 (or more) collocated measurement systems assuming none of them observed the true value. The method makes four assumptions about the measurement systems:

1. The signal and random errors are stationary (i.e., the mean of each is constant with time).
2. No cross-correlation of errors (i.e., measurement systems are independent of each other).
3. Error orthogonality (i.e., the measurement system errors are independent of the true value).
4. No error autocorrelation (i.e., the error estimates are not correlated with time).

TC commonly assumes an affine error model relating the observation to the true value and error:

$\boldsymbol{X}_i = \alpha_i + \beta_i \boldsymbol{t} + \boldsymbol{\varepsilon}_i$,

where $\boldsymbol{X}_i$ are the measured values from the $i$ collocated measurement systems, $\boldsymbol{t}$ are the measurements true values, and $\boldsymbol{\varepsilon}_i$ are the additive random errors.

There are two general methods/notations for approximating the error standard deviations (i.e., $\sigma_{\varepsilon,\ i}$): the **difference** notation and the **covariance** notation. While the two notations both approximate $\sigma_{\varepsilon,\ i}$, the distinction between the two is that the difference notation rescales the data sets to one of the data sets arbitrarily chosen as the reference data set, while the covariance notation does not. Since the difference notation approximates $\sigma_{\varepsilon,\ i}$ in the rescaled data space (which can be scaled back to the native data space), I only implement the covariance notation to reduce the work in having to rescale the data sets.

> There may be computational efficiencies to using the difference method. I can explore this later if needed.

### Covariance Notation

Solving for the covariance of two measurement systems gives

${\rm Cov}(\boldsymbol{X}_i, \boldsymbol{X}_j) = {\rm E}(\boldsymbol{X}_i, \boldsymbol{X}_j) - {\rm E}(\boldsymbol{X}_i){\rm E}(\boldsymbol{X}_j) = \beta_i \beta_j \sigma_\boldsymbol{t}^2 + \beta_i {\rm Cov}(\boldsymbol{t}, \boldsymbol{\varepsilon}_j) + \beta_j {\rm Cov}(\boldsymbol{t}, \boldsymbol{\varepsilon}_i) + {\rm Cov}(\boldsymbol{\varepsilon}_i, \boldsymbol{\varepsilon}_j)$.

With assumptions above, this simplifies to

${\rm Cov}(\boldsymbol{X}_i, \boldsymbol{X}_j) = \begin{cases} \beta_i \beta_j \sigma_{\boldsymbol{t}}^2, & {\rm for}\ i \ne j \\ \beta_i^2 \sigma_{\boldsymbol{t}}^2 + \sigma_{\varepsilon,\ i}^2, & {\rm for}\ i = j \end{cases}$,

since assumption 2 gives ${\rm Cov}(\boldsymbol{\varepsilon}_i \boldsymbol{\varepsilon}_j) = 0$ for $i \ne j$, and assumption 3 gives ${\rm Cov}(\boldsymbol{t}, \boldsymbol{\varepsilon}_i) = 0$.

Solving this system of equations is only possible with three or more measurement systems (to have at least the same number of equations as unknowns) and results in the following estimates of the error standard deviations:

> To actually solve this system requires setting $\beta_i \sigma_{\boldsymbol{t}} = \theta_i$ to reduce the unknowns by 1.

$\sigma_{\varepsilon,\ i}^2 = \sigma_{i}^2 - \frac{\sigma_{ij} \sigma_{ik}}{\sigma_{jk}}$,

where $\sigma_{ij} = {\rm Cov}(\boldsymbol{X}_i, \boldsymbol{X}_j)$.



In [1]:
def tc_covar(X):
    '''
        Uses the covariance method of Triple Collocation (TC) to estimate the error variances for the three collocated inputs.

        Parameters: X : array_like, shape(3, N)
                The three collocated inputs.

        Returns: evar : ndarray, shape(3)
                The estimated error variance of the three collocated inputs.
    '''
    if X is None:
        raise TypeError("tc_covar() missing 1 require positional argument: 'X'")
    
    import numpy as np
    
    X_array = np.array(X)
    
    # Error checking for shape and size
    if X_array.ndim != 2:
        raise ValueError('X must be a 2D array-like input. Current number of dimensions: %d' % X_array.ndim)
    if X_array.shape[0] != 3:
        raise ValueError('X must have a leading dimension of length 3. Current leading dimension length: %d' %X_array.shape[0])
    
    # Compute the covariance matrix of the inputs
    covar = np.cov(X_array)
    
    evar = np.array([covar[0, 0] - covar[0, 1] * covar[0, 2] / covar[1, 2],
                     covar[1, 1] - covar[0, 1] * covar[1, 2] / covar[0, 2],
                     covar[2, 2] - covar[0, 2] * covar[1, 2] / covar[0, 1]])
    
    return evar